<div class="head0">
    <div class="head0__name">
        Fokker-Planck solver
    </div>
    <div class="head0__note">
        Solution of the multidimensional Fokker-Planck equation by fast and accurate tensor based methods (basic examples).
    </div>
</div>

<div class="head1">
    <div class="head1__name">
        Export of the modules
    </div>
</div>

* **numpy** - standard library for numerical algebra;

* **ij** - function for jupyter styling

* **Grid** - class for multidimensional grid;

* **Model** - class for representation of equation;

* **Solver** - class with solver of the multidimensional Fokker-Planck equation;

* **Check** - utility class for checking results and testing.

In [1]:
import numpy as np

from fpcross import ij, Grid, Model, Solver, Check

ij()

Start |  1:21PM MSK on Dec 23, 2019 | python 3.7.1    |
-------------------------------------------------------


In [ ]:
from .mtr import difscheb, dif1cheb, dif2cheb

<div class="head1">
    <div class="head1__name">
        Solve equation using explicit functions
    </div>
    <div class="head1__note">
        Consider, for example, a 3-dimensional Focker-Planck equation with the zero drift (diffusion equation) and set the corresponding parameters by hands.
    </div>
</div>

<div class="head2">
    <div class="head2__name">
        Parameters
    </div>
    <div class="head2__note">
Consider
$$
    d x = f(x, t) \, dt + S(x, t) \, d \beta,
    \quad
    d \beta \, d \beta^{\top} = Q(t) dt,
    \quad
    x(0) = x_0 \sim \rho(x, 0) = \rho_0 (x),
$$
$$
    \frac{\partial \rho(x, t)}{\partial t} =
        \sum_{i=1}^d \sum_{j=1}^d
            \frac{\partial^2}{\partial x_i \partial x_j}
            \left[ D_{ij}(x, t) \rho(x, t) \right]
        - \sum_{i=1}^d
            \frac{\partial}{\partial x_i}
            \left[ f_i(x, t) \rho(x, t) \right],
    \quad
     D(x, t) = \frac{1}{2} S(x, t) Q(t) S(x, t)^{\top},
$$
where spatial $d$-dimensional ($d \ge 1$) variable $x \in R^d$ has probability density function (PDF) $\rho(x, t)$, $\beta$ is Brownian motion of dimension $q$ ($q \ge 1$, and we assume below that $q = d$), $f(x, t) \in R^d$ is a vector-function, $S(x, t) \in R^{d \times q}$ and $Q(t) \in R^{q \times q}$ are matrix-functions and $D(x, t) \in R^{d \times d}$ is a diffusion tensor.

Let
$$
    Q(t) \equiv I,
    \,
    S(x, t) \equiv \sqrt{2 D_c} I
    \implies
    D(x, t) \equiv D_c I,
$$
and
$$
    d = 3,
    \quad
    x \in \Omega,
    \quad
    \rho(x, t) |_{\partial \Omega} \approx 0,
    \quad
    f(x, t) \equiv 0,
    \quad
    \rho_0(x) = \frac{1}{(2 \pi s)^{\frac{3}{2}}}\exp{\left[-\frac{|x|^2}{2s}\right]}.
$$

It can be shown that the analytic solution is
$$
    \rho(x, t) =
        (2 \pi s + 4 \pi D t)^{-\frac{3}{2}}
        \exp{ \left[
            - \frac
                {
                    |x|^2
                }
                {
                    2  s + 4 D t
                }
        \right] },
$$
and the stationary solution ($t \rightarrow \infty$) is
$$
    \rho_{stat}(x) = 0.
$$
    </div>
</div>

<div class="note">
    Since interpolation is not required for the case of the zero drift ($f \equiv 0$), but our solver calculates it by design, then it is expected to operate much slower than another simple solvers.
</div>

In [5]:
s = 1.
D = 0.5

class Model_(Model):

    def d(self):
        ''' Spatial dimension. '''
        return 3
    
    def D(self):
        ''' Diffusion coefficient. '''
        return D

    def f0(self, X, t):
        ''' Function f(x, t). '''
        return np.zeros(X.shape)

    def f1(self, X, t):
        ''' Function f_i(x, t) / d x_i. (i = 1, 2, ..., d) '''
        return np.zeros(X.shape)

    def r0(self, X):
        ''' Initial condition. '''
        a = 2. * s
        r = np.exp(-np.sum(X*X, axis=0) / a) / (np.pi * a)**1.5
        return r.reshape(-1)

    def rt(self, X, t):
        ''' Exact analytic solution. '''
        a = 2. * s + 4. * D * t
        r = np.exp(-np.sum(X*X, axis=0) / a) / (np.pi * a)**1.5
        return r.reshape(-1)
    
    def with_rt(self):
        return True

<div class="note">
    
We can also set stationary solution $r_s(x)$, but in our case it is equal to zero. Since we do not want compare computation results with the zero, we do not define the corresponding function `rs(self, x)` and do not overwrite the function `with_rs(self)` in the model.
</div>

<div class="head2">
    <div class="head2__name">
        Solution in the dense (NP) format
    </div>
    <div class="head2__note"></div>
</div>

In [6]:
SL = Solver(
    TG=Grid(d=1, n=10, l=[+0., +1.], k='u'), # Time grid
    SG=Grid(d=3, n=21, l=[-5., +5.], k='c'), # Spatial grid
    MD=Model_()                              # Model for equation
)
SL.init()    # Init solver
SL.prep()    # Prepare special matrices
SL.calc()    # Solve equation
SL.TG.info() # Present results from all submodules
SL.SG.info()
SL.FN.info()
SL.info()

Solve: 100%|█| 9/9 [00:09<00:00,  1.12s/step, | At T=1.0e+00 :                                                                                                       Edert=5.0e-01  Ereal=4.5e-03]

------------------ Grid
Kind             : Uniform

Dimensions       : 1 
                 : Poi 10  | Min 0.000  | Max 1.000  | 
------------------ Grid
Kind             : Chebyshev

Dimensions       : 3 
                 : Poi 21  | Min -5.000 | Max 5.000  | 
------------------ Function  
Format           : 3D, NP--> Time (sec.)  |       
Prep             : 0.00e+00 
Calc             : 4.18e-03 
Comp (average)   : 0.00e+00 
Func (average)   : 0.00e+00 
------------------ Solver
Format    : 3D, NP [order=2]
Hst pois  : 10 
Hst with r: No 
d r / d t : 4.99e-01
Err  real : 4.47e-03
Time full : 1.21e+01 
Time prep : 2.23e+00 
Time calc : 9.90e+00 
    .init : 8.25e-03 
    .prep : 4.76e-05 
    .diff : 7.99e-01 
    .conv : 9.03e+00 
    .post : 5.30e-02 
    .last : 4.06e-03 



<div class="head2">
    <div class="head2__name">
        Solution in the sparse (TT) format
    </div>
    <div class="head2__note"></div>
</div>

In [7]:
SL = Solver(
    TG=Grid(d=1, n=10, l=[+0., +1.], k='u'), # Time grid
    SG=Grid(d=3, n=21, l=[-5., +5.], k='c'), # Spatial grid
    MD=Model_(),                             # Model for equation
    eps=1.E-2,                               # TT-approximation accuracy
    with_tt=True
)
SL.init()    # Init solver
SL.prep()    # Prepare special matrices
SL.calc()    # Solve equation
SL.info()

Solve:   0%|                                            | 0/9 [00:00<?, ?step/s]

TypeError: list indices must be integers or slices, not str

<div class="head1">
    <div class="head1__name">
        Solve equation, using the predefined model
    </div>
    <div class="head1__note">
        We can set the same equation, using the predefined corresponding model.
    </div>
</div>

In [6]:
MD = Model.select('fpe_3d_drift_zero')
MD.init(s=1., D=0.5).info(is_comp=True)

Model : fpe_3d_drift_zero      | d r(x,t) / d t = D \Delta r(x,t)
>>>>>>> Description            : Three-dimensional Focker Planck equation with the zero drift


<div class="head2">
    <div class="head2__name">
        Detailed info about model equation
    </div>
    <div class="head2__note">
        We can present the model description in markdown format.
    </div>
</div>

In [7]:
MD.info()

<div class="head0"><div class="head0__name">fpe_3d_drift_zero</div><div class="head0__note">Three-dimensional Focker Planck equation with the zero drift [FPE, 3D, analytic, time-diffusion].</div></div><div class="head2">
                <div class="head2__name">Parameters</div>
                <div class="head2__note"><ul><li>s =   1.0000 [Initial variance]<div>Variance of the initial condition (type: float, default:   1.0000)</div></li><li>D =   0.5000 [Diffusion coefficient]<div>Scalar diffusion coefficient (type: float, default:   0.5000)</div></li></ul></div>
            </div><div class="head1">
                <div class="head1__name">Description</div>
            </div>
Consider
$$
    d x = f(x, t) \, dt + S(x, t) \, d \beta,
    \quad
    d \beta \, d \beta^{\top} = Q(t) dt,
    \quad
    x(0) = x_0 \sim \rho(x, 0) = \rho_0 (x),
$$
$$
    \frac{\partial \rho(x, t)}{\partial t} =
        \sum_{i=1}^d \sum_{j=1}^d
            \frac{\partial^2}{\partial x_i \partial x_j}
            \left[ D_{ij}(x, t) \rho(x, t) \right]
        - \sum_{i=1}^d
            \frac{\partial}{\partial x_i}
            \left[ f_i(x, t) \rho(x, t) \right],
    \quad
     D(x, t) = \frac{1}{2} S(x, t) Q(t) S(x, t)^{\top},
$$
where spatial $d$-dimensional ($d \ge 1$) variable $x \in R^d$ has probability density function (PDF) $\rho(x, t)$, $\beta$ is Brownian motion of dimension $q$ ($q \ge 1$, and we assume below that $q = d$), $f(x, t) \in R^d$ is a vector-function, $S(x, t) \in R^{d \times q}$ and $Q(t) \in R^{q \times q}$ are matrix-functions and $D(x, t) \in R^{d \times d}$ is a diffusion tensor.

Let
$$
    Q(t) \equiv I,
    \,
    S(x, t) \equiv \sqrt{2 D_c} I
    \implies
    D(x, t) \equiv D_c I,
$$
and
$$
    d = 3,
    \quad
    x \in \Omega,
    \quad
    \rho(x, t) |_{\partial \Omega} \approx 0,
    \quad
    f(x, t) \equiv 0,
    \quad
    \rho_0(x) = \frac{1}{(2 \pi s)^{\frac{3}{2}}}\exp{\left[-\frac{|x|^2}{2s}\right]}.
$$

It can be shown that the analytic solution is
$$
    \rho(x, t) =
        (2 \pi s + 4 \pi D t)^{-\frac{3}{2}}
        \exp{ \left[
            - \frac
                {
                    |x|^2
                }
                {
                    2  s + 4 D t
                }
        \right] },
$$
and the stationary solution ($t \rightarrow \infty$) is
$$
    \rho_{stat}(x) = 0.
$$
        <div class="note">Since interpolation is not required for the case of the zero drift ($f \equiv 0$), but our solver calculates it by design, then it is expected to operate much slower than another simple solvers.</div><div class="end"></div>

<div class="head2">
    <div class="head2__name">
        Solution in the sparse (TT) format, using the model
    </div>
    <div class="head2__note"></div>
</div>

In [8]:
SL = Solver(
    TG=Grid(d=1, n=10, l=[+0., +1.], kind='u'),
    SG=Grid(d=3, n=21, l=[-5., +5.], kind='c'),
    MD=MD, eps=1.E-2, with_tt=True
)
SL.init()
SL.prep()
SL.calc()
SL.info()

Solve: 100%|███████| 9/9 [00:03<00:00,  2.69step/s, | At T=1.0e+00 : er=3.8e-03]

------------------ Solver
Format    : 3D, TT, eps= 1.00e-02 [order=2]
Time sec  : prep = 3.10e-03, calc = 2.93e+00
Err real  : 3.76e-03


<div class="head1">
    <div class="head1__name">
        Solution visualization
    </div>
    <div class="head1__note"></div>
</div>

In [9]:
# TODO!

<div class="head1">
    <div class="head1__name">
        Solution of the 3-dimensional Ornstein-Uhlenbeck process
    </div>
    <div class="head1__note">
        Let consider more complex example with known stationary solution.
    </div>
</div>

In [11]:
SL = Solver(
    TG=Grid(d=1, n=1000, l=[+0., +1.], kind='u'),
    SG=Grid(d=3, n= 31, l=[-3., +3.], kind='c'),
    MD=Model.select('fpe_oup').init(d=3, A=np.eye(3)),
    eps=1.E-6, with_tt=True
)
SL.init()
SL.prep()
SL.calc()
SL.MD.info()
SL.TG.info()
SL.SG.info()
SL.FN.info()
SL.info()

Solve: 100%|███| 999/999 [12:19<00:00,  1.33step/s, | At T=1.0e+00 : es=1.2e-01]


<div class="head0"><div class="head0__name">fpe-oup</div><div class="head0__note">Multidimensional Focker Planck equation (Ornstein–Uhlenbeck process) [FPE, ND, analytic, analyt-stationary, OUP].</div></div><div class="head2">
                <div class="head2__name">Parameters</div>
                <div class="head2__note"><ul><li>d =   3 [Dimension]<div>Spatial dimension (type: int, default:   1)</div></li><li>s =   1.0000 [Initial variance]<div>Variance of the initial condition (type: float, default:   1.0000)</div></li><li>D =   0.5000 [Diffusion coefficient]<div>Scalar diffusion coefficient (type: float, default:   0.5000)</div></li><li>A = [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]] [Drift]<div>Constant drift coefficients ([d x d] matrix) (type: ndarray, default: <function Model.pars.<locals>.<lambda> at 0x1513664ea0>)</div></li></ul></div>
            </div><div class="head1">
                <div class="head1__name">Description</div>
            </div>
Consider
$$
    d x = f(x, t) \, dt + S(x, t) \, d \beta,
    \quad
    d \beta \, d \beta^{\top} = Q(t) dt,
    \quad
    x(0) = x_0 \sim \rho(x, 0) = \rho_0 (x),
$$
$$
    \frac{\partial \rho(x, t)}{\partial t} =
        \sum_{i=1}^d \sum_{j=1}^d
            \frac{\partial^2}{\partial x_i \partial x_j}
            \left[ D_{ij}(x, t) \rho(x, t) \right]
        - \sum_{i=1}^d
            \frac{\partial}{\partial x_i}
            \left[ f_i(x, t) \rho(x, t) \right],
    \quad
     D(x, t) = \frac{1}{2} S(x, t) Q(t) S(x, t)^{\top},
$$
where spatial $d$-dimensional ($d \ge 1$) variable $x \in R^d$ has probability density function (PDF) $\rho(x, t)$, $\beta$ is Brownian motion of dimension $q$ ($q \ge 1$, and we assume below that $q = d$), $f(x, t) \in R^d$ is a vector-function, $S(x, t) \in R^{d \times q}$ and $Q(t) \in R^{q \times q}$ are matrix-functions and $D(x, t) \in R^{d \times d}$ is a diffusion tensor.

Let
$$
    Q(t) \equiv I,
    \,
    S(x, t) \equiv \sqrt{2 D_c} I
    \implies
    D(x, t) \equiv D I,
$$
and
$$
    \quad
    x \in \Omega,
    \quad
    \rho(x, t) |_{\partial \Omega} \approx 0,
    \quad
    f(x, t) = A (\mu - x),
    \quad
    \mu \equiv 0,
    \quad
    \rho_0(x) =
        \frac{1}{\left(2 \pi s \right)^{\frac{d}{2}}}
        \exp{\left[-\frac{|x|^2}{2s}\right]}.
$$

This equation has stationary solution ($t \rightarrow \infty$)
$$
    \rho_{stat}(x) =
        \frac
        {
            exp \left[ -\frac{1}{2} x^{\top} W^{-1} x \right]
        }
        {
            \sqrt{(2 \pi)^d det(W)}
        },
$$
where matrix $W$ is solution of the matrix equation
$$
    A W + W A^{\top} = 2 D.
$$
        <div class="note">The multivariate Ornstein–Uhlenbeck process is mean-reverting (the solution tends to its long-term mean $\mu$ as time $t$ tends to infinity) if if all eigenvalues of $A$ are positive and this process at any time is a multivariate normal random variable.</div><div class="note">We do not construct analytic solution for this multidimensional case, but use comparison with known stationary solution. The corresponding error will depend on the maximum value for the used time grid.</div><div class="end"></div>

------------------ Grid
Kind             : Uniform
Dimensions       : 1 
                 : Poi 1000 | Min 0.000  | Max 1.000  |
------------------ Grid
Kind             : Chebyshev
Dimensions       : 3 
                 : Poi 31  | Min -3.000 | Max 3.000  |
------------------ Function
Format           : 3D, TT, eps= 1.00e-06
--> Time         | 
Prep             : 0.00e+00 sec. 
Calc             : 5.73e-04 sec. 
Comp (average)   : 0.00e+00 sec. 
Func (average)   : 0.00e+00 sec. 
--> Cross params | 
nswp             :      200 
kickrank         :        1 
rf               : 2.00e+00 
--> Cross result | 
Func. evaluations:        0 
Cross iterations :        0 
Av. tt-rank      : 0.00e+00 
Cross err (rel)  : 0.00e+00 
Cross err (abs)  : 0.00e+00 
------------------ Solver
Format    : 3D, TT, eps= 1.00e-06 [order=2]
Time sec  : prep = 4.38e-03, calc = 6.98e+02
Err stat  : 1.19e-01


<div class="end"></div>